In [21]:
!pip install sentence_transformer pypdf faiss-gpu langchain-community
!pip install -U langchain-huggingface
!pip install langchain langchain-openai

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [42]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

In [43]:
# For openai key
import os
os.environ["OPENAI_API_KEY"] = ""


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [44]:
# loading the document (pdf)

loader = PyPDFLoader(file_path = '/content/drive/My Drive/qlore paper.pdf')
documents = loader.load()

In [45]:
# split the document in chunks

text = RecursiveCharacterTextSplitter().split_documents(documents)

In [46]:
# embedding the document
# embedding from hugging face

embeddings = HuggingFaceEmbeddings(model_name = 'BAAI/bge-small-en-v1.5', encode_kwargs = {"normalize_embeddings": True})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [47]:
# Similarity search and clustering dense vector using FAISS and develop vectorstore
vectorstore = FAISS.from_documents(text, embeddings)

vectorstore.save_local('/content/drive/MyDrive/vectorstore.db')

# create a retriever

retriever = vectorstore.as_retriever()


In [48]:
# load the llm (gpt 3.5 turbo)

llm =  ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0)

# prompt template
template = """
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

# create document chain
doc_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

In [49]:
# User query
response = chain.invoke({"input": "what is Qlora?"})

# Get the Answer only
response['answer']


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}